# Вариант 1


In [4]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

In [5]:
s = Singleton()
print("Object created", s)
s1 = Singleton()
print("Object created", s1)

Object created <__main__.Singleton object at 0x7f7028380bb0>
Object created <__main__.Singleton object at 0x7f7028380bb0>


# Вариант 2

In [6]:
class Singleton:
    __instance = None
    def __init__(self):
        if not Singleton.__instance:
            print(" __init__ method called..")
        else:
            print("Instance already created:", self.getInstance())
            
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton()
        return cls.__instance

In [8]:
s = Singleton() ## class initialized, but object not created
print("Object created", Singleton.getInstance()) # Object gets created
s1 = Singleton() ## instance already created

Instance already created: <__main__.Singleton object at 0x7f7028380a90>
Object created <__main__.Singleton object at 0x7f7028380a90>
Instance already created: <__main__.Singleton object at 0x7f7028380a90>


# Вариант 3

In [9]:
import sqlite3
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Database(metaclass=MetaSingleton):
    connection = None
    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj
    


In [10]:
db1 = Database().connect()
db2 = Database().connect()
print ("Database Objects DB1", db1)
print ("Database Objects DB2", db2)

Database Objects DB1 <sqlite3.Cursor object at 0x7f70282b00a0>
Database Objects DB2 <sqlite3.Cursor object at 0x7f70282b00a0>


# Вариант 4